# Spark Preparation
We check if we are in Google Colab.  If this is the case, install all necessary packages.

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.3.2 with hadoop 3.3, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.
Learn more from [A Must-Read Guide on How to Work with PySpark on Google Colab for Data Scientists!](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)

In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [10]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://dlcdn.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
    !tar xf spark-3.5.5-bin-hadoop3.tgz
    !mv spark-3.5.5-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

# Start a Local Cluster

In [11]:
import findspark
findspark.init()

In [12]:
spark_url = 'local'

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder.master(spark_url).appName('Netflix Rotten Tomatoes Metacritic IMDB').config('spark.ui.port', '4040').getOrCreate()

# Spark Assignment

Based on the movie review dataset in 'netflix-rotten-tomatoes-metacritic-imdb.csv', answer the below questions.

**Note:** do not clean or remove missing data

## What is the maximum and average of the overall hidden gem score?

In [47]:
file_path = '/content/netflix-rotten-tomatoes-metacritic-imdb.csv'

df = spark.read.option('header', True).csv(file_path)

In [48]:
df.show(5)

+-------------------+--------------------+--------------------+----------------+---------------+----------------+--------------------+------------+---------------+--------------------+--------------------+-----------+----------+---------------------+----------------+---------------+--------------------+----------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+
|              Title|               Genre|                Tags|       Languages|Series or Movie|Hidden Gem Score|Country Availability|     Runtime|       Director|              Writer|              Actors|View Rating|IMDb Score|Rotten Tomatoes Score|Metacritic Score|Awards Received|Awards Nominated For| Boxoffice|Release Date|Netflix Release Date|    Production House|        Netflix Link|           IMDb Link|             Summary|IMDb Votes|               Image|              

In [49]:
df.columns

['Title',
 'Genre',
 'Tags',
 'Languages',
 'Series or Movie',
 'Hidden Gem Score',
 'Country Availability',
 'Runtime',
 'Director',
 'Writer',
 'Actors',
 'View Rating',
 'IMDb Score',
 'Rotten Tomatoes Score',
 'Metacritic Score',
 'Awards Received',
 'Awards Nominated For',
 'Boxoffice',
 'Release Date',
 'Netflix Release Date',
 'Production House',
 'Netflix Link',
 'IMDb Link',
 'Summary',
 'IMDb Votes',
 'Image',
 'Poster',
 'TMDb Trailer',
 'Trailer Site']

In [46]:
from pyspark.sql.functions import max, avg

df.select(max('Hidden Gem Score')).show()

df.select(avg('Hidden Gem Score')).show()

+---------------------+
|max(Hidden Gem Score)|
+---------------------+
|                  9.8|
+---------------------+

+---------------------+
|avg(Hidden Gem Score)|
+---------------------+
|    5.937551386501226|
+---------------------+



## How many movies that are available in Korean Language?

In [56]:
from pyspark.sql.functions import col

df.filter(col('Languages').contains('Korea')).count()

735

## Which director has the highest average hidden gem score?

In [66]:
df.groupBy('Director').agg(avg('Hidden Gem Score').alias('avg_score')).orderBy('avg_score', ascending=False).show(1)

+-----------+---------+
|   Director|avg_score|
+-----------+---------+
|Dorin Marcu|      9.8|
+-----------+---------+
only showing top 1 row



## How many genres are there in the dataset?

In [77]:
from pyspark.sql.functions import split, explode

df.select(explode(split(col('Genre'), ', ')).alias('genre')).distinct().count()

28